# Предподготовка, очистка, фильтрация данных
# Извлечение полезной информации    


### В этом ноутбуке содержатся тренировочные материалы, решение задач с онлайн-курсов по обработке и анализу данных в Python

**Структура ноутбука**:  
1. Простая фильтрация и группировка данных по условию. Объединение таблиц  
3. Выгрузка из вложенных файлов. Объединение выгрузок
4. Фильтрация данных с помощью функции query и преобразование с apply и lambda-функциями  
5. Начальная обработка и извлечение полезной информации из датасета

In [616]:
# Загрузим необходимые библиотеки
import pandas as pd
import numpy as np
import os, re
from pprint import pprint
from datetime import timedelta

## 1. Простая фильтрация и группировка данных по условию. Объединение таблиц

### ЗАДАНИЕ 1.  
### Ответить на вопросы, применив различные способы фильтрации и группировки данных
В задании используется датасет с информацией о футболистах.

In [163]:
# Загрузим данные и посмотрим на них детальнее
football = pd.read_csv('data_sf.csv')
display(football.head())

,Unnamed: 0,Name,Age,Nationality,Club,Value,Wage,Position,Crossing,Finishing,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
0,0,L. Messi,31,Argentina,FC Barcelona,110500000,565000,RF,84,95,...,75,96,33,28,26,6,11,15,14,8
1,1,Cristiano Ronaldo,33,Portugal,Juventus,77000000,405000,ST,84,94,...,85,95,28,31,23,7,11,15,14,11
2,2,Neymar Jr,26,Brazil,Paris Saint-Germain,118500000,290000,LW,79,87,...,81,94,27,24,33,9,9,15,15,11
3,3,De Gea,27,Spain,Manchester United,72000000,260000,GK,17,13,...,40,68,15,21,13,90,85,87,88,94
4,4,K. De Bruyne,27,Belgium,Manchester City,102000000,355000,RCM,93,82,...,79,88,68,58,51,15,13,5,10,13


In [6]:
# Посмотрим на структуру датасета
display(football.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12897 entries, 0 to 12896
Data columns (total 42 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       12897 non-null  int64 
 1   Name             12897 non-null  object
 2   Age              12897 non-null  int64 
 3   Nationality      12897 non-null  object
 4   Club             12897 non-null  object
 5   Value            12897 non-null  int64 
 6   Wage             12897 non-null  int64 
 7   Position         12897 non-null  object
 8   Crossing         12897 non-null  int64 
 9   Finishing        12897 non-null  int64 
 10  HeadingAccuracy  12897 non-null  int64 
 11  ShortPassing     12897 non-null  int64 
 12  Volleys          12897 non-null  int64 
 13  Dribbling        12897 non-null  int64 
 14  Curve            12897 non-null  int64 
 15  FKAccuracy       12897 non-null  int64 
 16  LongPassing      12897 non-null  int64 
 17  BallControl      12897 non-null

None

In [9]:
# Выведем статистическую информацию
display(football.describe())
display(football.describe(include = 'object'))

,Unnamed: 0,Age,Value,Wage,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
count,12897.000000,12897.000000,1.289700e+04,12897.000000,12897.000000,12897.000000,12897.000000,12897.000000,12897.000000,12897.000000,...,12897.000000,12897.000000,12897.000000,12897.000000,12897.000000,12897.000000,12897.000000,12897.000000,12897.000000,12897.000000
mean,6605.040862,24.795379,2.112449e+06,7517.562224,47.076374,42.876328,50.202528,56.026983,40.295030,52.461735,...,46.459874,55.942932,45.655811,46.186710,44.386679,17.218345,16.962317,16.797938,16.950221,17.315965
std,3782.545526,4.872212,6.498424e+06,23061.985387,18.043470,19.096935,17.281593,14.740436,17.080687,18.871643,...,15.479313,11.280631,19.456346,21.172586,20.726546,18.085618,17.349624,16.971411,17.369297,18.335817
min,0.000000,16.000000,0.000000e+00,1000.000000,5.000000,2.000000,4.000000,7.000000,4.000000,4.000000,...,5.000000,12.000000,3.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,3357.000000,21.000000,2.500000e+05,1000.000000,35.000000,28.000000,43.000000,50.000000,28.000000,45.000000,...,37.000000,49.000000,29.000000,25.000000,23.000000,8.000000,8.000000,8.000000,8.000000,8.000000
50%,6622.000000,24.000000,5.000000e+05,2000.000000,51.000000,45.000000,54.000000,60.000000,41.000000,58.000000,...,47.000000,56.000000,51.000000,54.000000,51.000000,11.000000,11.000000,11.000000,11.000000,11.000000
75%,9876.000000,28.000000,8.500000e+05,4000.000000,61.000000,59.000000,62.000000,65.000000,53.000000,65.000000,...,58.000000,63.000000,62.000000,64.000000,62.000000,14.000000,14.000000,14.000000,14.000000,14.000000
max,13125.000000,45.000000,1.185000e+08,565000.000000,93.000000,95.000000,94.000000,93.000000,90.000000,97.000000,...,91.000000,96.000000,93.000000,93.000000,91.000000,90.000000,92.000000,91.000000,90.000000,94.000000


,Name,Nationality,Club,Position
count,12897,12897,12897,12897
unique,12326,156,650,27
top,J. Rodríguez,England,V-Varen Nagasaki,GK
freq,9,1368,30,1641


Итак, датафрейм football представляет собой таблицу с информацией о футболистах: имена и фамилии игроков, национальность, возраст, принадлежность к клубу, заработанная плата и различные спортивные показатели. Всего в наборе данных 42 колонки, в каждой из которых 12897 непустых значений. Пропуски в данных отсутствуют. В таблице 4 признака с типом данных object, остальные признаки имеют тип int. 

### Ответим на ряд вопросов, применив различные способы фильтрации и группировки данных

In [52]:
# Какова средняя скорость футболистов, зарплата которых выше среднего? Ответ округлите до сотых
# round(football.query('Wage > Wage.mean()').SprintSpeed.mean(),2)
display(round(football[football['Wage'] > football['Wage'].mean()]['SprintSpeed'].mean(),2))

# Какую позицию занимает футболист с самой высокой зарплатой?
display(football[football['Wage'] == football['Wage'].max()]['Position'].iloc[0])

# Сколько пенальти забили бразильские футболисты за период, данные о котором представлены в датасете?
display(football[football['Nationality'] == 'Brazil']['Penalties'].sum())

# Укажите средний возраст игроков, у которых точность удара головой > 50
display(round(football[football['HeadingAccuracy'] > 50]['Age'].mean(), 2))

# Укажите возраст самого молодого игрока, у которого хладнокровие и реакция превышают 90% от максимального значения, 
# представленного в датасете
display(football[(football['Composure'] > football['Composure'].max() * 0.9) & (football['Reactions'] > football['Reactions'].max() * 0.9)]['Age'].min())

# Определите, насколько средняя реакция самых взрослых игроков больше средней реакции самых молодых игроков
# round(football.query('Age == Age.max()').Reactions.mean() - football.query('Age == Age.min()').Reactions.mean(),2)
display(round((football[football['Age'] == football['Age'].max()]['Reactions'].mean()) -  (football[football['Age'] == football['Age'].min()]['Reactions'].mean()), 2))

67.57

'RF'

22789

25.59

24

22.64

In [72]:
# Из какой страны происходит больше всего игроков, чья стоимость превышает среднее значение?
display(football[football['Value'] > football['Value'].mean()]['Nationality'].value_counts().index[0])

# Определите, во сколько раз средняя зарплата голкипера с максимальным значением показателя " Рефлексы" 
# выше средней зарплаты голкипера с максимальным значением показателя "Владение мячом"
# round(football.query('(Position == "GK") & (GKReflexes == GKReflexes.max())').Wage.mean() / football.query('(Position == "GK") & (GKHandling == GKHandling.max())').Wage.mean(), 2)
display(round(football[(football['Position'] == 'GK') & (football['GKReflexes'] == football['GKReflexes'].max())]['Wage'].mean() / football[(football['Position'] == 'GK') & (football['GKHandling'] == football['GKHandling'].max())]['Wage'].mean(), 2))

# Определите, во сколько раз средняя сила удара самых агрессивных игроков выше средней силы удара игроков с минимальной агрессией
#  round(football.query('Aggression == Aggression.max()').ShotPower.mean() / football.query('Aggression == Aggression.min()').ShotPower.mean(), 2)
display(round(football[football['Aggression'] == football['Aggression'].max()]['ShotPower'].mean() / football[football['Aggression'] == football['Aggression'].min()]['ShotPower'].mean(), 2))

# Данные об игроках каких позиций занимают более 10% датасета?
# football.groupby('Position').Name.count()[football.groupby('Position').Name.count() > len(football.Position) * 0.1].index.tolist()
pos_2 = football['Position'].value_counts(normalize = True)
display(pos_2[pos_2 > 0.1].index.tolist())

# Данные об игроках каких позиций занимают менее 1% датасета?
#  football.groupby('Position').Name.count()[football.groupby('Position').Name.count() < len(football.Position) * 0.01]
display(football.Position.value_counts(normalize = True)[football.Position.value_counts(normalize = True) < 0.01].index.tolist())

# В каких пределах находятся худшие 20% показателей точности ударов ногой?
display(football['FKAccuracy'].value_counts(bins = 5).index.min())

'Spain'

2.77

2.08

['GK', 'ST', 'CB']

['LS', 'RS', 'RWB', 'LWB', 'CF', 'LF', 'LAM', 'RF', 'RAM']

Interval(2.908, 21.2, closed='right')

In [97]:
# У какого процента испанских футболистов зарплата находится в пределах 25 % минимума 
# от наблюдаемого уровня зарплат среди испанских игроков?
wage_proportion = football[football['Nationality'] == 'Spain']['Wage'].value_counts(bins = 4, normalize = True)
display(round(wage_proportion[sorted(wage_proportion.index)[0]], 2))

# Укажите количество национальностей, к которым относятся футболисты, выступающие за клуб "Manchester United"
# len(football[football.Club == 'Manchester United'].Nationality.unique())
# len(football[football.Club == 'Manchester United'].Nationality.value_counts())
display(football[football['Club'] == 'Manchester United']['Nationality'].nunique())

# Определите двух футболистов из Бразилии, выступающих за клуб 'Juventus'
display(football[(football['Nationality'] == 'Brazil') & (football['Club'] == 'Juventus')][['Name']])

# Укажите, какие два клуба насчитывают наибольшее количество футболистов возрастом старше 35 лет
display(football[football.Age > 35].Club.value_counts().sort_values(ascending = False)[:2])
display(football[football.Age > 35].groupby('Club').Age.count().sort_values(ascending = False).index.tolist()[:2])

# Решение с помощью isin
small_data = football[football['Age'] > 35]['Club'].value_counts()
display(small_data[small_data.isin([small_data.max()])])     # small_data.index[:2]

0.97

13

,Name
64,Alex Sandro
65,Douglas Costa


Hokkaido Consadole Sapporo    4
Club Atlético Huracán         4
Name: Club, dtype: int64

['Club Atlético Huracán', 'Hokkaido Consadole Sapporo']

Hokkaido Consadole Sapporo    4
Club Atlético Huracán         4
Name: Club, dtype: int64

In [154]:
# Разбейте всех футболистов родом из Аргентины на 4 равные группы по возрасту. 
# Сколько футболистов из Аргентины в возрасте от 34,75 до 41 года?
argentina_players_age = football[football['Nationality'] == 'Argentina']['Age'].value_counts(bins = 4)
display(argentina_players_age[argentina_players_age.index.left >= 34.75])

# Сколько процентов футболистов из Испании имеют возраст 21 год? 
# Введите с точностью до 2 знаков после запятой без указания знака % (например, 12.35)
# round(football[(football.Age == 21) & (football.Nationality == 'Spain')].Name.nunique() / len(football[football.Nationality == 'Spain']),2)
display(round(football[football['Nationality'] == 'Spain']['Age'].value_counts(normalize = True)[21], 2))

# Определите позиции, по которым общая сумма зарплат игроков превышает 5 млн евро в год
display(football.groupby(['Position'])['Wage'].sum()[football.groupby(['Position'])['Wage'].sum() > 5000000].index.tolist())
display(football.groupby(['Position'])['Wage'].sum()[football.groupby(['Position'])['Wage'].sum() > 5000000])

# Посчитайте среднюю зарплату и цену игроков разных позиций. Представители какой позиции имеют самую высокую среднюю ценность? 
# Какова средняя зарплата футболистов на данной позиции?
display(football.groupby('Position', as_index = False)[['Wage', 'Value']].mean().head(3))
display(football.groupby('Position', as_index = False)[['Wage', 'Value']].mean().sort_values(by = 'Value', ascending = False).iloc[[0]])

# football.groupby(['Position'])['Wage'].mean().loc['LF']
# football.groupby(football['Position'] == 'LF')['Wage'].mean()[1]
display(football[football.Position == 'LF'].Wage.mean())

(34.75, 41.0]    49
Name: Age, dtype: int64

0.12

['CAM', 'CB', 'CM', 'GK', 'LB', 'LM', 'RB', 'RM', 'ST']

Position
CAM     5500000
CB      7476000
CM      6194000
GK      8784000
LB      6096000
LM      5440000
RB      5034000
RM      5098000
ST     11873000
Name: Wage, dtype: int64

,Position,Wage,Value
0,CAM,8688.783570,2.963199e+06
1,CB,5328.581611,1.321251e+06
2,CDM,7121.562952,1.658900e+06


,Position,Wage,Value
11,LF,75125.0,27362500.0


75125.0

In [159]:
# Посчитайте среднюю и медианную зарплату футболистов из разных клубов. 
# В скольких клубах средняя и медианная зарплаты совпадают?
football_2 = football.groupby('Club', as_index = False) \
        .agg(mean_wage = ('Wage','mean'), median_wage = ('Wage','median'))
display(football_2.head(3))

# len(football_2[football_2.mean_wage == football_2.median_wage])
# football_2[football_2.mean_wage == football_2.median_wage].Club.nunique()
display(football_2[football_2.mean_wage == football_2.median_wage].Club.count())

# ДРУГОЙ ВАРИАНТ ГРУППИРОВКИ
football_new = football.groupby('Club').Wage.agg(func = ['mean', 'median']).reset_index() \
                .rename(columns = {'mean' : 'mean_wage', 'median': 'median_wage'})
display(football_new.head(3))
display(len(football_new[football_new.mean_wage == football_new.median_wage]) )


# Продолжаем работать с клубами, в которых средняя зарплата совпадает с медианной. 
# Каков максимальный размер средней зарплаты в этой группе клубов?
equals = football_2[football_2.mean_wage == football_2.median_wage]
display(equals.head(3))
display(equals['mean_wage'].max())

# Как называется клуб, где игроки получают такую зарплату?
# equals.sort_values('mean_wage', ascending = False)[:1]
# equals[equals['mean_wage'] == equals['mean_wage'].max()]
display(equals[equals.mean_wage.isin([equals.mean_wage.max()])])

,Club,mean_wage,median_wage
0,SSV Jahn Regensburg,3750.000000,3500.0
1,1. FC Heidenheim 1846,3800.000000,3000.0
2,1. FC Kaiserslautern,1434.782609,1000.0


52

,Club,mean_wage,median_wage
0,SSV Jahn Regensburg,3750.000000,3500.0
1,1. FC Heidenheim 1846,3800.000000,3000.0
2,1. FC Kaiserslautern,1434.782609,1000.0


52

,Club,mean_wage,median_wage
4,1. FC Magdeburg,4000.0,4000.0
12,AEK Athens,1000.0,1000.0
15,AJ Auxerre,2000.0,2000.0


13000.0

,Club,mean_wage,median_wage
168,Cruzeiro,13000.0,13000.0


In [210]:
# Посчитайте сумму зарплат футболистов клуба "Chelsea"
# football.groupby('Club')['Wage'].sum()['Chelsea']
# football.groupby(football['Club'] == "Chelsea")['Wage'].sum()[1]
display(football[football.Club == 'Chelsea'].Wage.sum())

# Определите минимальную и максимальную зарплату футболиста национальности Аргентина в возрасте 20 лет
# football.query('(Nationality == "Argentina") & (Age == 20)').agg(min_wage = ('Wage', 'min'), max_wage = ('Wage', 'max'))
display(football[(football.Nationality == 'Argentina') & (football.Age == 20)].agg(min_wage = ('Wage', 'min'), max_wage = ('Wage', 'max')))

# Определите минимальную и максимальную зарплату футболиста национальности Аргентина в возрасте 30 лет
# football.query('(Nationality == "Argentina") & (Age == 30)').agg(min_wage = ('Wage', 'min'), max_wage = ('Wage', 'max'))
display(football[(football.Nationality == 'Argentina') & (football.Age == 30)].agg(min_wage = ('Wage', 'min'), max_wage = ('Wage', 'max')))

# Определите максимальную силу и баланс среди игроков клуба "FC Barcelona" из Аргентины
# football.query("(Nationality == 'Argentina') & (Club == 'FC Barcelona')")[['Strength', 'Balance']].max()
# football[(football['Club'] == "FC Barcelona") & (football['Nationality'] == "Argentina")][['Strength', 'Balance']].max()
display(football.groupby((football['Club'] == "FC Barcelona") & (football['Nationality'] == "Argentina"))[['Strength', 'Balance']].max().loc[1])

2035000

,Wage
min_wage,1000
max_wage,54000


,Wage
min_wage,1000
max_wage,300000


Strength    59
Balance     95
Name: True, dtype: int64

In [283]:
# Создайте сводную таблицу, содержащую сведения о количестве игроков, занимающих разные позиции в каждом клубе. 
# Отсутствующие значения замените нулями. Каково среднее количество вратарей (GK) по всем клубам?

# Сводная таблица с мультииндексами:
pivot_1 = football.pivot_table(index = ['Club'], columns = ['Position'], values = ['Name'], aggfunc = 'count', fill_value = 0)
display(pivot_1.head(3))
display(pivot_1.index[:3])
display(pivot_1.columns[:3])
print(f'Среднее количество вратарей (GK) по всем клубам: {round(pivot_1["Name", "GK"].mean(), 2)}')


# Сводная таблица с привычными индексами:
pivot_2 = football.pivot_table(index = 'Club', columns = 'Position', values = 'Name', aggfunc = 'count', fill_value = 0)
display(pivot_2.head(3))
display(pivot_2.index[:3])
display(pivot_2.columns[:3])
print(f'Среднее количество вратарей (GK) по всем клубам: {round(pivot_2["GK"].mean(), 2)}')

# Используя таблицу, созданную на предыдущем шаге, определите, сколько клубов не содержат данных о центральных полузащитниках 
# (CM)
# pivot_1['Name', 'CM'].value_counts().loc[0]
# pivot_1[pivot_1['Name']['CM'] == 0].reset_index().Club.nunique()
# pivot_1[pivot_1['Name']['CM'] == 0]['Name']['CM'].count()
display(len(pivot_1[pivot_1['Name', 'CM'] == 0]))
display(len(pivot_2[pivot_2['CM'] == 0]))

# С помощью сводной таблицы посчитайте, сколько получают русские футболисты, играющие за ФК "AS Monaco"
pivot_3 = football.loc[football['Nationality'].isin(['Russia'])].pivot_table(index = 'Club', columns = 'Nationality', 
                                                                             values = 'Wage', aggfunc = 'sum') 
display(pivot_3.head(3))
display(pivot_3.loc["AS Monaco"])

# ДРУГОЙ ВАРИАНТ РЕШЕНИЯ
pivot_4 = football.pivot_table(index = 'Nationality', columns = 'Club', values = 'Wage', 
                               aggfunc = 'sum', fill_value = 0).reset_index()
pivot_4[pivot_4.Nationality == 'Russia']["AS Monaco"]

Name                                 ...                 \
Position               CAM CB CDM CF CM GK LAM LB LCB LCM  ... RB RCB RCM RDM   
Club                                                       ...                  
 SSV Jahn Regensburg     0  5   1  0  2  3   0  2   0   0  ...  2   1   0   1   
1. FC Heidenheim 1846    1  3   0  0  1  2   0  2   1   0  ...  2   1   0   0   
1. FC Kaiserslautern     0  3   3  0  0  3   0  2   1   0  ...  2   0   0   1   

                                          
Position              RF RM RS RW RWB ST  
Club                                      
 SSV Jahn Regensburg   0  0  0  0   0  3  
1. FC Heidenheim 1846  0  3  0  0   0  2  
1. FC Kaiserslautern   0  2  1  0   0  1  

[3 rows x 27 columns]

Index([' SSV Jahn Regensburg', '1. FC Heidenheim 1846',
       '1. FC Kaiserslautern'],
      dtype='object', name='Club')

MultiIndex([('Name', 'CAM'),
            ('Name',  'CB'),
            ('Name', 'CDM')],
           names=[None, 'Position'])

Среднее количество вратарей (GK) по всем клубам: 2.52


Position,CAM,CB,CDM,CF,CM,GK,LAM,LB,LCB,LCM,...,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
Club,,,,,,,,,,,,,,,,,,,,,
SSV Jahn Regensburg,0,5,1,0,2,3,0,2,0,0,...,2,1,0,1,0,0,0,0,0,3
1. FC Heidenheim 1846,1,3,0,0,1,2,0,2,1,0,...,2,1,0,0,0,3,0,0,0,2
1. FC Kaiserslautern,0,3,3,0,0,3,0,2,1,0,...,2,0,0,1,0,2,1,0,0,1


...

136

136

Nationality,Russia
Club,
AS Monaco,61000
Cracovia,1000
Dalkurd FF,1000


Nationality
Russia    61000
Name: AS Monaco, dtype: int64

122    61000
Name: AS Monaco, dtype: int64

In [321]:
# Создайте сводную таблицу, содержащую сведения о средней скорости футболистов, 
# занимающих разные позиции в разных футбольных клубах
pivot_5 = football.pivot_table(index = 'Club', columns = 'Position', values = 'SprintSpeed', aggfunc = 'mean')
display(pivot_5.head(3))

# Основываясь на данных таблицы, выделите три позиции, представители которых обладают самой высокой скоростью
# pivot_5.loc[:].max().sort_values(ascending = False).index.tolist()[:3]
display(pivot_5.max().sort_values(ascending = False)[:3])

# Используя таблицу, созданную на предыдущем шаге, выделите названия трёх клубов, в которых центральные форварды (ST) 
# обладают наибольшей средней скоростью
# pivot_5['ST'].sort_values(ascending = False).index.tolist()[:3]
display(pivot_5.reset_index().sort_values(by = 'ST', ascending = False)[['Club', 'ST']][:3])

Position,CAM,CB,CDM,CF,CM,GK,LAM,LB,LCB,LCM,...,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
Club,,,,,,,,,,,,,,,,,,,,,
SSV Jahn Regensburg,NaN,46.600000,82.000000,NaN,61.0,33.000000,NaN,76.0,NaN,NaN,...,68.5,57.0,NaN,65.0,NaN,NaN,NaN,NaN,NaN,61.666667
1. FC Heidenheim 1846,68.0,61.333333,NaN,NaN,68.0,41.000000,NaN,60.0,55.0,NaN,...,80.5,59.0,NaN,NaN,NaN,78.333333,NaN,NaN,NaN,69.000000
1. FC Kaiserslautern,NaN,59.000000,65.666667,NaN,NaN,42.666667,NaN,68.0,45.0,NaN,...,71.5,NaN,NaN,55.0,NaN,82.500000,69.0,NaN,NaN,79.000000


Position
LM    94.0
RS    94.0
RM    94.0
dtype: float64

Position,Club,ST
621,Vitória,91.0
571,Sydney FC,88.0
111,CD Lugo,87.0


### ЗАДАНИЕ 2.   
### Ответить на вопросы, применив различные способы объединения таблиц  
В задании используется датасет с логами рекомендательной системы фильмов. Основой данных будут две таблицы. 

**Таблица ratings —  данные о выставленных оценках фильмов. Содержит признаки:**  
- **userId** — идентификатор пользователя, который поставил оценку фильму
- **movieId** — идентификатор фильма
- **rating** — выставленная оценка
- **timestamp** — время (в формате unix time), когда была выставлена оценка

**Таблица movies — расшифровка идентификаторов фильмов.Содержит признаки:**  
- **movieId** — идентификатор фильма
- **title** — название фильма
- **genre** — список жанров, к которым относится фильм

In [324]:
# Загрузим данные и посмотрим на них детальнее
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

display(ratings.head(3))
display(movies.head(3))

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [325]:
# Посмотрим на структуру датасетов
display(ratings.info())
display(movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


None

### Ответим на ряд вопросов, применив различные способы объединения таблиц

In [341]:
# Объедините датафреймы ratings и movies. Какой год выпуска у фильма с movieId=3456?
merged_df = movies.merge(ratings, on = 'movieId', how = 'outer')
display(merged_df.head(3))
display(merged_df[merged_df['movieId'] == 3456]['title'])

# Сколько раз была выставлена самая низкая оценка 0.5 в рейтингах?
# merged_df[merged_df.rating.isin([merged_df['rating'].min()])].movieId.count()
display(merged_df['rating'].min())
display(merged_df[merged_df['rating'] == merged_df['rating'].min()]['movieId'].count())

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09


54467    Color of Paradise, The (Rang-e khoda) (1999)
Name: title, dtype: object

0.5

1370

### ЗАДАНИЕ 3.   
### Ответить на вопросы, используя данные о  новых поступлениях интернет-магазина.   
В словаре items_dict содержится информация о наличии товара на складе. В словаре purchase_log — данные о покупках товаров.

In [348]:
items_dict = {'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], 
              'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
              'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]}
purchase_log = {'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
                'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], 
                'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]}
print(items_dict)
print()
print(purchase_log)

{'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], 'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'], 'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]}

{'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221], 'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], 'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]}


Переведем словари в датафремы и посмотрим на признаки:  
- **item_id** — идентификатор модели (по этому столбцу будем объединять датафреймы)
- **vendor** — производитель модели
- **stock_count** — имеющееся на складе количество данных моделей (в штуках)
- **purchase_id** — идентификатор покупки
- **price** — стоимость модели в покупке

In [350]:
items = pd.DataFrame(items_dict)
purchase = pd.DataFrame(purchase_log)
display(items)
display(items.info())

,item_id,vendor,stock_count
0,417283,Samsung,54
1,849734,LG,33
2,132223,Apple,122
3,573943,Apple,18
4,19475,LG,102
5,3294095,Apple,43
6,382043,Samsung,77
7,302948,Samsung,143
8,100132,LG,60
9,312394,ZTE,19


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   item_id      10 non-null     int64 
 1   vendor       10 non-null     object
 2   stock_count  10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 368.0+ bytes


None

In [351]:
display(purchase)
display(purchase.info())

,purchase_id,item_id,price
0,101,417283,13900
1,101,849734,5330
2,101,132223,38200
3,112,573943,49990
4,121,19475,9890
5,145,3294095,33000
6,145,382043,67500
7,145,302948,34500
8,145,103845,89900
9,221,100132,11400


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   purchase_id  10 non-null     int64
 1   item_id      10 non-null     int64
 2   price        10 non-null     int64
dtypes: int64(3)
memory usage: 368.0 bytes


None

In [374]:
# Объедините датафреймы. Определите, модель с каким item_id есть в статистике продаж, но не учтена на складе
new_df = items.merge(purchase, on = 'item_id', how = 'outer')
display(new_df.head(3))
# new_df = new_df.fillna(0)
# new_df[(new_df['stock_count'] == 0) & (new_df['purchase_id'] != 0)]['item_id']

id = new_df[(new_df.stock_count.isna()) & (~new_df.purchase_id.isna())].item_id.iloc[0]
print(f'Модель с item_id = {id} есть в статистике продаж, но не учтена на складе.')

# Сформируйте датафрейм merged, в котором в результате объединения останутся модели, которые учтены на складе и имели продажи. 
# Сколько всего таких моделей?
merged = items.merge(purchase, on = 'item_id', how = 'inner')
print(f'Кол-во моделей, которые учтены на складе и имели продажи: {len(merged)}')

# Посчитайте объем выручки для каждой модели, которую можно получить, распродав все остатки на складе. 
# Модель с каким item_id имеет максимальное значение выручки после распродажи остатков?
total_revenue = new_df['stock_count'] * new_df['price']
display(total_revenue)
max_revenue_ind = total_revenue[total_revenue == total_revenue.max()].index[0]
print(f'Индекс модели с максимальным значениес выручки после распродажи остатков: {max_revenue_ind}')
print(f'Идентификатор модели (item_id): {new_df["item_id"][max_revenue_ind]}')

# Посчитайте итоговую выручку из прошлого задания по всем моделям
print(f'Итоговая выручка по всем моделям: {total_revenue.sum()}')

,item_id,vendor,stock_count,purchase_id,price
0,417283,Samsung,54.0,101.0,13900.0
1,849734,LG,33.0,101.0,5330.0
2,132223,Apple,122.0,101.0,38200.0


Модель с item_id = 103845 есть в статистике продаж, но не учтена на складе.
Кол-во моделей, которые учтены на складе и имели продажи: 9


0      750600.0
1      175890.0
2     4660400.0
3      899820.0
4     1008780.0
5     1419000.0
6     5197500.0
7     4933500.0
8      684000.0
9           NaN
10          NaN
dtype: float64

Индекс модели с максимальным значениес выручки после распродажи остатков: 6
Идентификатор модели (item_id): 382043
Итоговая выручка по всем моделям: 19729490.0


## 3. Выгрузка из вложенных файлов. Объединение выгрузок

### ЗАДАНИЕ 1.  
### Напишите цикл, который собирает содержимое файлов папки data_22 в единый датафрейм data. Сколько строк в датафрейме data?
В папке data_22 содержится набор из 10 файлов формата txt. Это файл ratings.csv, разбитый на 10 частей.

In [386]:
files = os.listdir('data_22')
files

['ratings_1.txt',
 'ratings_10.txt',
 'ratings_2.txt',
 'ratings_3.txt',
 'ratings_4.txt',
 'ratings_5.txt',
 'ratings_6.txt',
 'ratings_7.txt',
 'ratings_8.txt',
 'ratings_9.txt']

In [377]:
for root, dirs, files in os.walk('data_22'):
    print(root, dirs, files)

data_22 [] ['ratings_1.txt', 'ratings_10.txt', 'ratings_2.txt', 'ratings_3.txt', 'ratings_4.txt', 'ratings_5.txt', 'ratings_6.txt', 'ratings_7.txt', 'ratings_8.txt', 'ratings_9.txt']


In [384]:
data = pd.DataFrame(columns = ['userId','movieId','rating','timestamp'])
display(data)

,userId,movieId,rating,timestamp


In [385]:
count_temp = 0
for filename in files:
    print(filename)
    temp = pd.read_csv(os.path.join('data_22', filename), names = ['userId','movieId','rating','timestamp'])
    count_temp += temp.shape[0]
    
    data = pd.concat([data, temp])
    print(data.shape)
    
data.shape[0] == count_temp

ratings_1.txt
(9999, 4)
ratings_10.txt
(20004, 4)
ratings_2.txt
(30004, 4)
ratings_3.txt
(40004, 4)
ratings_4.txt
(50004, 4)
ratings_5.txt
(60004, 4)
ratings_6.txt
(70004, 4)
ratings_7.txt
(80004, 4)
ratings_8.txt
(90004, 4)
ratings_9.txt
(100004, 4)


True

In [388]:
display(data.head())
print(data.shape)
display(data.info())

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


(100004, 4)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 100004 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  object 
 1   movieId    100004 non-null  object 
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  object 
dtypes: float64(1), object(3)
memory usage: 3.8+ MB


None

## 4. Фильтрация данных с помощью функции query и преобразование c apply и lambda-функциями

### ЗАДАНИЕ 1.  
### Ответить на вопросы, применив query, apply и lambda-функцию
В задании используется тренировочный датасет с информацией о профессиональной принадлежности людей.

In [440]:
# Загрузим данные и посмотрим на них детальнее
sample = pd.read_csv('sample.csv')
display(sample.head(3))
display(sample.info())
display(sample.describe(include = ['object']))
print(f'Кол-во пропущенных значений: \n{sample.isna().sum()[sample.isna().sum() > 0]}')
print(f'\nКол-во уникальных значений: \n{sample.nunique()}')

,Name,City,Age,Profession
0,Иванов,Москва,25,Клинер
1,NaN,Волгоград,31,Менеджер
2,Иванов,Москва,25,Клинер


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        10 non-null     object
 1   City        10 non-null     object
 2   Age         11 non-null     int64 
 3   Profession  11 non-null     object
dtypes: int64(1), object(3)
memory usage: 480.0+ bytes


None

,Name,City,Profession
count,10,10,11
unique,9,9,4
top,Иванов,Москва,Рабочий
freq,2,2,5


Кол-во пропущенных значений: 
Name    1
City    1
dtype: int64

Кол-во уникальных значений: 
Name           9
City           9
Age           10
Profession     4
dtype: int64


В датасете sample:  
- Есть пропущенные значения  
- Прочерк вместо фамилии  
- Есть фамилии, написанные латиницей

In [441]:
# Приведите названия колонок к нижнему регистру для удобства обращения
sample.columns = sample.columns.str.lower()

# Выведите информацию о людях в возрасте меньше 30 лет
# sample[sample['age'] < 30]
sample.query('age < 30')

# Выведите информацию о людях, проживающих в городах: Сочи, Рига
# sample.query('city in ["Рига", "Сочи"]')
sample[(sample.city.str.contains('Со', na = False)) | (sample.city.str.contains('Ри', na = False))]

# Найдите всех людей, фамилии которых начинаются на букву "К"
# sample.query('name.str.contains("К", na = False)')
k_surnames = sample[sample['name'].str.match('К', na = False)]
display(k_surnames)

# Найдите всех людей, фамилии которых начинаются на буквы, отличные от "К"
# sample.query('~ name.str.contains("K", na = False)')
sample[~ sample['name'].str.match('К', na = False)]

# Найдите записи, где в городах есть / нет буквы «о»
# sample.query('city.str.contains("о", na = False)')
sample[sample.city.str.contains('о', na = False)]
sample[~ sample.city.str.contains('о', na = False)]

# Замените все буквы в поле City на маленькие
sample.city = sample.city.apply(lambda x: x.lower() if not pd.isna(x) else np.nan)

# Напишите функцию profession_code, которая на вход получает строку, а на выход возвращает:
# 0 — если на вход поступила строка "Рабочий", 1 — если на вход поступила строка "Менеджер", 2 — в любом другом случае
# Замените поле profession
def profession_code(x):
    return 0 if x == "Рабочий" else 1 if x == "Менеджер" else 2

sample['profession'] = sample['profession'].apply(profession_code)
display(sample.head(3))

# Напишите функцию age_category, которая на вход получает число, а возвращает:
# "молодой" — если возраст меньше 23, "средний" — если возраст от 23 до 35, "зрелый" — если возраст больше 35
# Cоздайте новую колонку в sample под названием 'age_category'
def age_category(x):
    return "молодой" if x < 23 else "средний" if 23 <= x <= 35 else "зрелый"
sample['age_category'] = sample.age.apply(age_category)
display(sample.head(3))

,name,city,age,profession
5,Кузнецов,Сочи,19,Рабочий
7,Котлеревский,Йошкар-Ола,32,Рабочий


,name,city,age,profession
0,Иванов,москва,25,2
1,NaN,волгоград,31,1
2,Иванов,москва,25,2


,name,city,age,profession,age_category
0,Иванов,москва,25,2,средний
1,NaN,волгоград,31,1,средний
2,Иванов,москва,25,2,средний


## 5. Начальная обработка и извлечение полезной информации из датасета

### ЗАДАНИЕ 1.  
### Ответить на вопросы, провести некоторые этапы предобработки данных
В задании используется два датасета c информацией о посещении букмекерской конторы  

**1. Таблица 'log.csv' содержит следующие столбцы:**  
  - Идентификатор пользователя  
  - Время посещения  
  - Дата посещения  
  - Размер ставки  
  - Размер выигрыша  

**2. Таблица 'users.csv' содердит следующие столбцы:**
  - Идентификатор пользователя
  - Адрес электронной почты
  - Город

**Проведем начальную предобработку данных и ответим на ряд вопросов:**  
- Сколько раз клиенту нужно прийти в букмекерскую контору, чтобы сделать ставку?  
- Каков средний выигрыш в процентах?  
- Каков баланс по каждому пользователю?  
- Какие города самые выгодные?  
- В каких городах самая высокая ставка?  
- Сколько времени проходит от первого посещения клиентом сайта до первой ставки?

In [567]:
# Загрузить данные, используя дополнительные параметры 
logs = pd.read_csv('log.csv', header = None)
users = pd.read_csv('users.csv', encoding = 'KOI8-R', sep = '\t')
display(logs.head(3))
display(users.head(3))

,0,1,2,3
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,NaN,NaN
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN


,Юзверь,мейл,Гео
0,User_943,Accumanst@gmail.com,Ижевск
1,User_908,Advismowr@mail.ru,Ижевск
2,User_962,Anachso@ukr.net,Краснодар


In [568]:
# Переименуем колонки и посмотрим детальнее на данные
logs.columns = ['user_id', 'time', 'bet', 'win']
users.columns = ['user_id', 'email', 'geo']

display(logs)
display(logs.info())
print(f'\nРазмер датасета: {logs.shape}')
print('\nСтатистическая информация:')
display(logs.describe())
display(logs.describe(include = 'object'))
print(f'\nКоличество пропущенных значений:\n{logs.isna().sum()}')
print(f'\nКоличество уникальных значений:\n{logs.nunique()}')

,user_id,time,bet,win
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,NaN,NaN
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN
3,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,NaN
4,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,NaN
...,...,...,...,...
995,Запись пользователя № - user_984,[2019-04-20 9:59:58,9754.0,NaN
996,#error,NaN,10054.0,29265.0
997,#error,NaN,10454.0,NaN
998,#error,NaN,1000.0,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   user_id  1000 non-null   object 
 1   time     985 non-null    object 
 2   bet      485 non-null    float64
 3   win      138 non-null    float64
dtypes: float64(2), object(2)
memory usage: 31.4+ KB


None


Размер датасета: (1000, 4)

Статистическая информация:


,bet,win
count,4.850000e+02,1.380000e+02
mean,6.785738e+03,9.562102e+04
std,7.720292e+04,8.554669e+05
min,1.000000e+02,3.691000e+03
25%,3.000000e+02,5.096250e+03
50%,6.000000e+02,5.935000e+03
75%,9.000000e+02,6.978750e+03
max,1.678321e+06,9.876543e+06


,user_id,time
count,1000,985
unique,102,984
top,Запись пользователя № - user_966,[2019-03-04 21:03:54
freq,18,2



Количество пропущенных значений:
user_id      0
time        15
bet        515
win        862
dtype: int64

Количество уникальных значений:
user_id    102
time       984
bet         36
win        136
dtype: int64


In [569]:
display(users)
display(users.info())
print(f'\nРазмер датасета: {users.shape}')
print('\nСтатистическая информация:')
display(users.describe())
print(f'\nКоличество пропущенных значений:\n{users.isna().sum()}')
print(f'\nКоличество уникальных значений:\n{users.nunique()}')

,user_id,email,geo
0,User_943,Accumanst@gmail.com,Ижевск
1,User_908,Advismowr@mail.ru,Ижевск
2,User_962,Anachso@ukr.net,Краснодар
3,User_973,Antecia@inbox.ru,Пермь
4,User_902,Balliaryva@ukr.net,NaN
...,...,...,...
95,User_959,UpdatesCurious@yahoo.com,Тюмень
96,User_901,V2artierso@mail.ru,Арзангелтск
97,User_970,Vashoterlo@bk.ru,Воронеж
98,User_965,Visuareda@yahoo.com,Воронеж


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  100 non-null    object
 1   email    99 non-null     object
 2   geo      97 non-null     object
dtypes: object(3)
memory usage: 2.5+ KB


None


Размер датасета: (100, 3)

Статистическая информация:


,user_id,email,geo
count,100,99,97
unique,100,99,14
top,User_943,Accumanst@gmail.com,Санкт-Петербург
freq,1,1,10



Количество пропущенных значений:
user_id    0
email      1
geo        3
dtype: int64

Количество уникальных значений:
user_id    100
email       99
geo         14
dtype: int64


In [570]:
users['geo'].unique()

array(['Ижевск', 'Краснодар', 'Пермь', nan, 'Санкт-Петербург',
       'Ярославль', 'Екатеринбург', 'Ставрополь', 'Красноярск', 'Воронеж',
       'Арзангелтск', 'Москва', 'Хабаровск', 'Тюмень', 'Казань'],
      dtype=object)

Данные содержат большое количество пропусков, лишних символов "[" и технических ошибок "#error". 

### Применим ряд преобразований и ответим на вопросы

In [571]:
# Создайте новый датафрейм log_win, в который будут входить только записи, где пользователь выиграл. 
# Посчитайте, сколько таких записей, и сохраните в переменной win_count.
log_win = logs[logs['win'] > 0]
win_count = log_win['win'].count()   # len(log_win)
print(win_count)

# Найдите тех, у кого ставка меньше 2000, а выигрыш больше 0
# logs[(logs.bet < 2000) & (logs.win >0)]
logs.query('(bet < 2000) & (win > 0)').head(3)

# Удалите записи, в которых есть ошибка в поле user_id
# logs = logs[logs['user_id'] != '#error']
# logs = logs.user_id[~logs.user_id.str.match('#error')]
logs = logs[~logs.user_id.str.contains('#er', na = False)]

# Преобразуйте поле user_id в датафрейме logs, оставив только идентификатор пользователя
# logs.user_id.str.replace('Запись пользователя № - ', '')
# logs.user_id.apply(lambda x: x.replace('Запись пользователя № - ', ''))
logs['user_id'] = logs['user_id'].str.split(' - ').apply(lambda x: x[1])

# Удалите квадратную скобку из строк в столбце time, чтобы привести его к более привычному формату
# logs.time = logs['time'].str.replace('[', '')
# logs.time.apply(lambda x: x.replace('[', ""))
logs['time'] = logs['time'].apply(lambda x: np.nan if pd.isna(x) else x.replace('[', ''))
display(logs)

138


,user_id,time,bet,win
0,user_919,2019-01-01 14:06:51,NaN,NaN
1,user_973,2019-01-01 14:51:16,NaN,NaN
2,user_903,2019-01-01 16:31:16,NaN,NaN
3,user_954,2019-01-01 17:17:51,NaN,NaN
4,user_954,2019-01-01 21:31:18,NaN,NaN
...,...,...,...,...
991,user_965,2019-04-20 12:55:41,800.0,6927.0
992,user_967,2019-04-20 14:59:36,10154.0,NaN
993,user_973,2019-04-20 17:09:56,10254.0,NaN
994,user_977,2019-04-20 18:10:07,10354.0,NaN


In [572]:
# Преобразуйте признак time к datetime. После этого найдите наибольшую дату
logs.time = pd.to_datetime(logs.time)
print(logs.time.max())

# Найдите месяц, который встречался в данных реже всего
# logs.time.apply(lambda x: x.month).value_counts().sort_values()[:1]
less_frequent_month = logs.time.dt.month.value_counts(ascending = False).index[-1]
print(less_frequent_month)

# Посчитайте, сколько дней в данных являются выходными (т.е. сб или вск)
# logs.time.dt.dayofweek.value_counts()[[5,6]].sum()
# logs.time.dt.dayofweek[logs.time.dt.dayofweek.isin([5,6])].count()
# logs.time[logs.time.dt.dayofweek.apply(lambda x: x in [5,6])].count()
display(logs.time.apply(lambda x: x.dayofweek).value_counts()[[5,6]].sum())

# Напишите функцию, которая преобразует дату во время дня
# С 0 до 5 часов - ночь, с 6 до 11 - утро, с 12 до 17 - день, с 18 до 23 - вечер.
# Посчитайте, какое время дня встречается в данных реже всего
def time_of_day(x):
    return 'Ночь' if 0 <= x <= 5 else 'Утро' if 6 <= x <= 11 else 'День' if 12 <= x <= 17 else 'Вечер'
logs.time.dt.hour.apply(time_of_day).value_counts().sort_values()[:1]

# Посчитайте, сколько раз люди приходили, но не делали ставок 
logs.bet = logs.bet.fillna(0)
bet_null = logs.bet[logs.bet == 0].value_counts().iloc[0]  # logs[logs.bet == 0].user_id.count()
print(bet_null) 

2019-04-20 18:10:07
4


283

513


#### Поработаем с признаком win, в котором также есть пропуски.  

Предположим, что если в признаке win находится пропуск, то выигрыша не было. Здесь два возможных случая:  

- Человек не делал ставки и ничего не выиграл. Т.е. просто пришел, посмотрел и ушел.  
- Человек сделал ставку, но не выиграл. Значит, выигрыш на самом деле является отрицательным значением - это проигрыш.  

Напишите метод, который заполнит пропуски в признаке win в соответствии с предположением выше. 

Для этого можно подготовить функцию, которая вычисляет размер выигрыша (или проигрыша) по следующей схеме:  

- Если значение в столбце win существует (не пропуск) - вернуть это же значение. Это значит, что человек выиграл.  
- Если вместо значения в столбце win и в столбце bet пропуски, вернуть 0.  
- Если в столбце bet нет пропуска, а в столбце win есть пропуск, вернуть отрицательное значение столбца bet (проигрыш).  

На выходе получится таблица без пропусков в столбце win.

In [573]:
logs

,user_id,time,bet,win
0,user_919,2019-01-01 14:06:51,0.0,NaN
1,user_973,2019-01-01 14:51:16,0.0,NaN
2,user_903,2019-01-01 16:31:16,0.0,NaN
3,user_954,2019-01-01 17:17:51,0.0,NaN
4,user_954,2019-01-01 21:31:18,0.0,NaN
...,...,...,...,...
991,user_965,2019-04-20 12:55:41,800.0,6927.0
992,user_967,2019-04-20 14:59:36,10154.0,NaN
993,user_973,2019-04-20 17:09:56,10254.0,NaN
994,user_977,2019-04-20 18:10:07,10354.0,NaN


In [575]:
def fillna_win(df):
    if not pd.isna(df['win']):
        return df['win']
    elif all([(pd.isna(df['win'])), (not df['bet'])]):
        return 0
    elif all([(pd.isna(df['win'])), (df['bet'] > 0)]):
        return -df['bet']

# def fillna_win(row): 
#     if np.isnan(row['win']) and not row['bet']:   
#         return 0
#     elif np.isnan(row['win']) and row['bet']:      
#         return -row['bet']
#     elif row['win']:
#         return row['win']  


# Замените старое значение столбца win на новый
logs.win = logs.apply(lambda row: fillna_win(row), axis = 1)
display(logs)

# Посчитайте, сколько раз участники ставок проиграли деньги
print(logs.win.loc[logs.win < 0].count())  # len(logs[logs.win < 0])

#### 14. Создайте признак net, хранящий сумму выигрыша с учетом ставки.  
Для этого используйте следующий алгоритм:  
- если значение признака win меньше 0 - присвоить значение признака win признаку net;  
- во всех остальных случаях - из значения признака win вычтите значение признака bet и полученное значение присвойте признаку net.  

После этого посчитайте, у скольких людей выигрыш положительный.

In [583]:
# Создайте признак net, хранящий сумму выигрыша с учетом ставки. После этого посчитайте, у скольких людей выигрыш положительный
def fill_net(row):
    return row['win'] if row['win'] < 0 else row['win'] - row['bet']

logs['net'] = logs.apply(lambda row: fill_net(row), axis = 1)
display(logs[logs['net'] > 0]['net'].count() )  # len(logs[logs.net > 0])

# Посчитайте среднее и медианное значение выигрыша в тех случаях, когда выигрыш больше 0. Результат округлите до целого
display(round(logs[logs.net > 0]['net'].mean()))
display(round(logs[logs.net > 0]['net'].median()))

133

82626

5329

In [585]:
# Посчитайте, какой процент посещений букмекерской конторы оборачивался ставкой
# round(logs[logs.bet > 0].user_id.count() / logs.bet.count(), 2)
display(round(len(logs[logs.bet > 0]) / len(logs.user_id), 2))

# Посчитайте среднее значение ставки в тех случаях, когда ставка была сделана
display(round(logs[logs.bet > 0]['bet'].mean()))

# Посчитайте средний выигрыш в тех случаях, когда ставка была сделана. Результат округлите до целого
display(round(logs[logs['bet'] > 0]['net'].mean()))

# Посчитайте среднее значение потерь при проигрыше. Результат округлите до целого
# Ответ должен быть дан в виде отрицательного числа
display(round(logs[logs.net < 0].net.mean()))

# Посчитайте, какой процент ставок заканчивается выигрышем, а какой - проигрышем 
# Сравните эти значения и ответьте, какое из них больше
display(round(logs[logs['net'] > 0]['net'].count() / logs[logs['bet'] > 0]['bet'].count(), 2)) # процент выигрышей
display(round(logs[logs['net'] < 0]['net'].count() / logs[logs['bet'] > 0]['bet'].count(), 2)) # процент проигрышей

0.48

6807

20879

-3346

0.28

0.72

Результаты показывают, что процент ставок, заканчивающихся проигрышем, гораздо выше процента ставок, заканчивающихся выигрышем.

In [607]:
# Чему была равна минимальная ставка и сколько раз сделали такую ставку
min_bet = logs[logs['bet'] != 0]['bet'].min()
display(min_bet)
# logs[logs.bet == min_bet]['user_id'].count()
display(logs[logs.bet.isin([min_bet])].user_id.count())

# Сколько уникальных игороков сделали минимальную ставку 
# logs.user_id[logs.bet == min_bet].nunique()
logs[logs.bet.isin([min_bet])].user_id.nunique()

# Объедините таблицы logs и users
users.user_id = users.user_id.apply(lambda x: x.lower())
bookie = pd.merge(users, logs, on = 'user_id')
display(bookie.head(3))

100.0

48

,user_id,email,geo,time,bet,win,net
0,user_943,Accumanst@gmail.com,Ижевск,2019-01-06 21:06:57,0.0,0.0,0.0
1,user_943,Accumanst@gmail.com,Ижевск,2019-01-10 12:16:22,100.0,-100.0,-100.0
2,user_943,Accumanst@gmail.com,Ижевск,2019-01-27 01:18:47,100.0,-100.0,-100.0
3,user_943,Accumanst@gmail.com,Ижевск,2019-02-10 11:09:24,100.0,5456.0,5356.0
4,user_943,Accumanst@gmail.com,Ижевск,2019-02-12 15:09:38,300.0,5245.0,4945.0
...,...,...,...,...,...,...,...
970,user_965,Visuareda@yahoo.com,Воронеж,2019-04-20 12:55:41,800.0,6927.0,6127.0
971,user_921,Aavast@ya.ru,Ижевск,2019-01-07 01:05:59,0.0,0.0,0.0
972,user_921,Aavast@ya.ru,Ижевск,2019-02-05 14:08:57,0.0,0.0,0.0
973,user_921,Aavast@ya.ru,Ижевск,2019-02-09 23:20:21,400.0,6067.0,5667.0


In [622]:
# Используйте объединенный датасет. Посчитайте медиану баланса по всем пользователям
display(bookie.groupby(['user_id'])['net'].sum().median())

# Сколько раз в среднем каждый человек приходит, не делая ставок, при условии, 
# что у этого человека все-таки есть хотя бы одна ставка?
bet_0 = bookie[bookie['bet'] == 0].groupby(['user_id'])['bet'].count()   # количество нулевых ставок по гурппам
bet_1 = bookie[bookie['bet'] > 0].groupby(['user_id'])['bet'].count()    # количество не нулевых ставок по гурппам
print(bet_0.sum() / len(bet_1))

# Сколько в среднем времени проходит между появлением человека в сервисе и первой ставкой?
bet_0_min_time = bookie[bookie['bet'] == 0].groupby(['user_id'])['time'].min() # минимальная дата среди ставок равных нулю
bet_over_0_min_time = bookie[bookie['bet'] > 0].groupby(['user_id'])['time'].min() # минимальная дата среди ставок больше нуля
user_bet_all = pd.merge(bet_0_min_time, bet_over_0_min_time, on ='user_id', how = 'right') # объединим две таблицы в одну
user_bet_all['difference'] = user_bet_all['time_y'] - user_bet_all['time_x']  # создадим столбец с разницей между датами 
shift = user_bet_all['difference'].sum() / len(bookie[bookie['bet'] > 0].groupby(['user_id']).count())
print(shift)
# user_bet_all['difference'].sum() / user_bet_all.__len__()

# Наибольший суммарный выигрыш среди всех городов имеет Москва. Посчитайте следующий за ней город
display(bookie.groupby(['geo'])['win'].sum().sort_values(ascending = False).index[1])

# Во сколько раз различаются максимальное и минимальное значение средней ставки по городам?
display(round(bookie[bookie['bet'] != 0].groupby(['geo'])['bet'].mean().max() / bookie[bookie['bet'] != 0].groupby(['geo'])['bet'].mean().min()))

1986.0

5.05
46 days 06:54:48


'Воронеж'

128

**Итак, на этапе начальной предобработки удалось выявить несколько интересных моментов, которые в перспективе могут принести пользу бизнесу:**  
- Реже всего люди заходят в букмекерскую контору вечером.  
- Около половины раз люди приходили, но не делали ставок.  
- А те, кто делал ставки, проиграли больше чем в 2/3 случаев.  
- Средняя ставка и средний проигрыш примерно равны по абсолютному значению.  
- Может пройти довольно много времени между первым приходом человека и первой ставкой (46 дней). И за это время человек может зайти в контору еще несколько раз, не делая ставок.  
- Интересный факт: второй по величине город России и второй по суммарному выигрышу на ставках город России - разные города (Санкт-Петербург и Воронеж).